# **Homework 2 Phoneme Classification**

* Slides: https://docs.google.com/presentation/d/1v6HkBWiJb8WNDcJ9_-2kwVstxUWml87b9CnA16Gdoio/edit?usp=sharing
* Kaggle: https://www.kaggle.com/c/ml2022spring-hw2
* Video: TBA


### Preparing Data

**Helper functions to pre-process the training data from raw MFCC features of each utterance.**

A phoneme may span several frames and is dependent to past and future frames. \
Hence we concatenate neighboring phonemes for training to achieve higher accuracy. The **concat_feat** function concatenates past and future k frames (total 2k+1 = n frames), and we predict the center frame.

Feel free to modify the data preprocess functions, but **do not drop any frame** (if you modify the functions, remember to check that the number of frames are the same as mentioned in the slides)

In [1]:
import os
import random
import pandas as pd
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.utils import rnn
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
from tqdm import tqdm

def load_feat(path):
    feat = torch.load(path)
    return feat

def preprocess_data(split, feat_dir, phone_path, train_ratio=0.8, train_val_seed=1337):
    class_num = 41 # NOTE: pre-computed, should not need change
    mode = 'train' if (split == 'train' or split == 'val') else 'test'

    label_dict = {}
    if mode != 'test':
      phone_file = open(os.path.join(phone_path, f'{mode}_labels.txt')).readlines()

      for line in phone_file:
          line = line.strip('\n').split(' ')
          label_dict[line[0]] = [int(p) for p in line[1:]]

    if split == 'train' or split == 'val':
        # split training and validation data
        usage_list = open(os.path.join(phone_path, 'train_split.txt')).readlines()
        random.seed(train_val_seed)
        random.shuffle(usage_list)
        percent = int(len(usage_list) * train_ratio)
        usage_list = usage_list[:percent] if split == 'train' else usage_list[percent:]
    elif split == 'test':
        usage_list = open(os.path.join(phone_path, 'test_split.txt')).readlines()
    else:
        raise ValueError('Invalid \'split\' argument for dataset: PhoneDataset!')

    usage_list = [line.strip('\n') for line in usage_list]
    print('[Dataset] - # phone classes: ' + str(class_num) + ', number of utterances for ' + split + ': ' + str(len(usage_list)))

    seqlen = []
    maxlen = 4000
    X = torch.empty(4000, 1000, 39)
    if mode != 'test':
      y = torch.empty(4000, 1000, dtype=torch.long)

    idx = 0
    for i, fname in tqdm(enumerate(usage_list)):
        feat = load_feat(os.path.join(feat_dir, mode, f'{fname}.pt'))
        cur_len = len(feat)
        if mode != 'test':
          label = torch.LongTensor(label_dict[fname])
        X[idx, 0:cur_len, :] = feat
        if mode != 'test':
          y[idx, 0:cur_len] = label
        seqlen.append(cur_len)
        idx += 1

    X = X[:idx, :]
    if mode != 'test':
      y = y[:idx]

    print(f'[INFO] {split} set')
    print(X.shape)
    if mode != 'test':
      print(y.shape)
      return X, torch.tensor(seqlen, dtype=torch.int64), y
    else:
      return X, torch.tensor(seqlen, dtype=torch.int64)


In [2]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
print(f'DEVICE: {device}')

DEVICE: cuda:0


## Define Model

In [9]:
class BasicBlock(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(BasicBlock, self).__init__()

        self.block = nn.Sequential(
            nn.Linear(input_dim, output_dim),
            nn.ReLU(),
            nn.BatchNorm1d(output_dim),
            nn.Dropout(p=0.2),
        )

    def forward(self, x):
        x = self.block(x)
        return x

class interm(nn.Module):
    def __init__(self, hidden_size, **kwargs):
        super(interm, self).__init__(**kwargs)
        self.hs = hidden_size
    def forward(self, X):
        data, seqlen = rnn.pad_packed_sequence(X[0], batch_first=True, total_length=1000)
        idx = 0
        for j in range(seqlen.shape[0]):
            idx += seqlen[j]
        dest = torch.empty(idx, self.hs, device=device)
        idx = 0
        for j in range(seqlen.shape[0]):
            dest[idx:idx + seqlen[j],:] = data[j, 0:seqlen[j],:]
            idx += seqlen[j]
        return dest

class Classifier(nn.Module):
    def __init__(self, output_dim=41):
        super(Classifier, self).__init__()
        hidden_size = 256
        bi = True
        dup = 2
        self.fc = nn.Sequential(
            nn.LSTM(39, hidden_size, dropout=0.2, num_layers=4, batch_first=True, bidirectional=bi),
            interm(hidden_size * dup),
            nn.BatchNorm1d(hidden_size * dup),
            nn.ReLU(),
            nn.Dropout(0.3),
            BasicBlock(hidden_size * 2, 64),
            nn.Linear(64, output_dim)
        )

    def forward(self, x):
        x = self.fc(x)
        return x

## Hyper-parameters

In [4]:
# data prarameters
pad_length = 1000               # the number of frames to concat with, n must be odd (total 2k+1 = n frames)
train_ratio = 0.9               # the ratio of data used for training, the rest will be used for validation

# training parameters
seed = 114514                   # random seed
batch_size = 64                 # batch size
num_epoch = 50                  # the number of training epoch
learning_rate = 0.002            # learning rate
model_path = './model.ckpt'     # the path where the checkpoint will be saved

## Prepare dataset and model

In [5]:
import gc

# preprocess data
train_X, train_seqlen, train_y = preprocess_data(split='train', feat_dir='/kaggle/input/ml2022spring-hw2/libriphone/libriphone/feat', phone_path='/kaggle/input/ml2022spring-hw2/libriphone/libriphone', train_ratio=train_ratio)
val_X, val_seqlen, val_y = preprocess_data(split='val', feat_dir='/kaggle/input/ml2022spring-hw2/libriphone/libriphone/feat', phone_path='/kaggle/input/ml2022spring-hw2/libriphone/libriphone', train_ratio=train_ratio)

# get dataset
train_set = TensorDataset(*(train_X.to(device), train_seqlen, train_y.to(device)))
val_set = TensorDataset(*(val_X.to(device), val_seqlen, val_y.to(device)))

# remove raw feature to save memory
del train_X, train_y, val_X, val_y
gc.collect()

# get dataloader
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False)

[Dataset] - # phone classes: 41, number of utterances for train: 3857


3857it [00:32, 118.95it/s]


[INFO] train set
torch.Size([3857, 1000, 39])
torch.Size([3857, 1000])
[Dataset] - # phone classes: 41, number of utterances for val: 429


429it [00:03, 114.81it/s]


[INFO] val set
torch.Size([429, 1000, 39])
torch.Size([429, 1000])


In [6]:
#fix seed
def same_seeds(seed):
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)  
    np.random.seed(seed)  
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

In [10]:
# fix random seed
same_seeds(seed)

# create model, define a loss function, and optimizer
model = Classifier().to(device)
criterion = nn.CrossEntropyLoss() 
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

best_acc = 0.0

## Training

In [12]:
for epoch in range(num_epoch):
    train_acc = 0.0
    train_loss = 0.0
    val_acc = 0.0
    val_loss = 0.0
    
    # training
    model.train() # set the model to training mode
    train_items = 0
    for i, batch in enumerate(tqdm(train_loader)):
        features, seqlen, labels = batch
        
        optimizer.zero_grad() 
        outputs = model(rnn.pack_padded_sequence(features, seqlen, batch_first=True, enforce_sorted=False))
        
        idx = 0
        dest = torch.empty(batch_size * 1000, dtype=torch.int64, device=device)
        for j in range(seqlen.shape[0]):
            dest[idx:idx + seqlen[j]] = labels[j, 0:seqlen[j]]
            idx += seqlen[j]
        labels = dest[:idx]
        train_items += labels.shape[0]
        
        loss = criterion(outputs, labels)
        loss.backward() 
        optimizer.step() 
        
        _, train_pred = torch.max(outputs, 1) # get the index of the class with the highest probability
        train_acc += (train_pred.detach() == labels.detach()).sum().item()
        train_loss += loss.item()
    
    # validation
    val_items = 0
    if len(val_set) > 0:
        model.eval() # set the model to evaluation mode
        with torch.no_grad():
            for i, batch in enumerate(tqdm(val_loader)):
                features, seqlen, labels = batch
                outputs = model(rnn.pack_padded_sequence(features, seqlen, batch_first=True, enforce_sorted=False))
                
                idx = 0
                dest = torch.empty(batch_size * 1000, dtype=torch.int64, device=device)
                for j in range(seqlen.shape[0]):
                    dest[idx:idx + seqlen[j]] = labels[j, 0:seqlen[j]]
                    idx += seqlen[j]
                labels = dest[:idx];
                val_items += labels.shape[0]
                
                loss = criterion(outputs, labels)
                _, val_pred = torch.max(outputs, 1) 
                val_acc += (val_pred.cpu() == labels.cpu()).sum().item() # get the index of the class with the highest probability
                val_loss += loss.item()

            print('[{:03d}/{:03d}] Train Acc: {:3.6f} Loss: {:3.6f} | Val Acc: {:3.6f} loss: {:3.6f}'.format(
                epoch + 1, num_epoch, train_acc/train_items, train_loss/train_items, val_acc/val_items, val_loss/val_items
            ))
            print(val_acc)
            # if the model improves, save a checkpoint at this epoch
            if val_acc > best_acc:
                best_acc = val_acc
                torch.save(model.state_dict(), model_path)
                print('saving model with acc {:.3f}'.format(best_acc/val_items))
    else:
        print('[{:03d}/{:03d}] Train Acc: {:3.6f} Loss: {:3.6f}'.format(
            epoch + 1, num_epoch, train_acc/train_items, train_loss/train_items
        ))

# if not validating, save the last epoch
if len(val_set) == 0:
    torch.save(model.state_dict(), model_path)
    print('saving model at last epoch')


100%|██████████| 7/7 [00:01<00:00,  4.12it/s]


[001/050] Train Acc: 0.962413 Loss: 0.000003 | Val Acc: 0.830570 loss: 0.000024
219744.0


100%|██████████| 7/7 [00:01<00:00,  4.12it/s]


[002/050] Train Acc: 0.966733 Loss: 0.000002 | Val Acc: 0.833167 loss: 0.000025
220431.0
saving model with acc 0.833


100%|██████████| 7/7 [00:01<00:00,  4.11it/s]


...

[031/050] Train Acc: 0.981421 Loss: 0.000001 | Val Acc: 0.834501 loss: 0.000030
220784.0


100%|██████████| 7/7 [00:01<00:00,  4.05it/s]


[032/050] Train Acc: 0.981481 Loss: 0.000001 | Val Acc: 0.832608 loss: 0.000030
220283.0


  3%|▎         | 2/61 [00:03<01:31,  1.55s/it]


KeyboardInterrupt: 

In [ ]:
del train_loader, val_loader
gc.collect()

## Testing
Create a testing dataset, and load model from the saved checkpoint.

In [13]:
# load data
test_X, test_seqlen = preprocess_data(split='test', feat_dir='/kaggle/input/ml2022spring-hw2/libriphone/libriphone/feat', phone_path='/kaggle/input/ml2022spring-hw2/libriphone/libriphone')
test_set = TensorDataset(test_X, test_seqlen)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)

[Dataset] - # phone classes: 41, number of utterances for test: 1078


1078it [00:09, 110.39it/s]

[INFO] test set
torch.Size([1078, 1000, 39])


In [14]:
# load model
model = Classifier().to(device)
model.load_state_dict(torch.load(model_path))

<All keys matched successfully>

Make prediction.

In [15]:
test_acc = 0.0
test_lengths = 0
pred = np.array([], dtype=np.int32)

model.eval()
with torch.no_grad():
    for i, batch in enumerate(tqdm(test_loader)):
        features, seqlen = batch
        features = features.to(device)

        outputs = model(rnn.pack_padded_sequence(features, seqlen, batch_first=True, enforce_sorted=False))

        _, test_pred = torch.max(outputs, 1) # get the index of the class with the highest probability
        pred = np.concatenate((pred, test_pred.cpu().numpy()), axis=0)


100%|██████████| 17/17 [00:04<00:00,  3.95it/s]


Write prediction to a CSV file.

After finish running this block, download the file `prediction.csv` from the files section on the left-hand side and submit it to Kaggle.

In [19]:
with open('prediction.csv', 'w') as f:
    f.write('Id,Class\n')
    for i, y in enumerate(pred):
        f.write('{},{}\n'.format(i, y))